In [1]:
y = 'Success_High'
start_from = 1 # Lowest is 1

In [2]:
import pandas as pd
import warnings
import time
warnings.filterwarnings("ignore")
sme=pd.read_excel("/kaggle/input/sme-data-final-v11/SME_data_final_v11.xlsx")

In [3]:
sme

,mapping_key,url,Issuer Company,Company Name,subscription_link,NSE_symbol,Close Year,Close Year Previous,1 day before Close Day,1 week before Close Day week number,...,answer_of_question_16,answer_of_question_17,answer_of_question_18,answer_of_question_19,answer_of_question_20,answer_of_question_21,answer_of_question_22,answer_of_question_23,answer_of_question_24,answer_of_question_25
0,906,https://www.chittorgarh.com/ipo/nitiraj-engine...,nitiraj engineers,Nitiraj Engineers Ltd IPO,https://www.chittorgarh.com/ipo_subscription/n...,NITIRAJ,2017,2016,2017-02-28,8,...,The potential impact of market fluctuations on...,The IPO price of ₹40 per equity share has been...,"The IPO price is ₹40/- per Equity Share, which...",Based on the information provided in the Red H...,"Based on the Red Herring Prospectus (RHP), the...","Based on the provided content, the company, Me...","Yes, Mega Flex Plastics Limited has certified ...",The relevant content from the Red Herring Pros...,"Based on the contents provided, the company's ...",The shareholding pattern of Mega Flex Plastics...
1,190,https://www.chittorgarh.com/ipo/sanginita-chem...,sanginita chemicals,Sanginita Chemicals Limited IPO,https://www.chittorgarh.com/ipo_subscription/s...,SANGINITA,2017,2016,2017-03-02,8,...,"Based on the contents provided, the potential ...",Based on the provided Red Herring Prospectus (...,Based on the provided contents from the Red He...,The IPO price of ₹ 100 per share is 10 times t...,"Based on the Red Herring Prospectus (RHP), the...","According to the content, the company operates...","Based on the given contents, it appears that t...",Based on the provided content from the Red Her...,"Based on the Red Herring Prospectus (RHP), the...","Based on the provided contents, the shareholdi..."
2,793,https://www.chittorgarh.com/ipo/rmc-switchgear...,rmc switchgears,RMC Switchgears Limited IPO,https://www.chittorgarh.com/ipo_subscription/r...,NaN,2017,2016,2017-03-02,8,...,"Based on the contents provided, the potential ...",Based on the Red Herring Prospectus (RHP) of R...,"Based on the provided content, it is difficult...",Based on the Red Herring Prospectus (RHP) of R...,"Based on the contents provided, the company, R...",Based on the Red Herring Prospectus (RHP) of R...,"Based on the contents provided, the company, R...",Based on the provided Red Herring Prospectus (...,Based on the Red Herring Prospectus (RHP) of R...,"Based on the contents provided, the shareholdi..."
3,124,https://www.chittorgarh.com/ipo/akash-infra-pr...,akash infra projects,Akash Infra Projects Ltd IPO,https://www.chittorgarh.com/ipo_subscription/a...,AKASH,2017,2016,2017-03-05,9,...,"Based on the contents provided, the potential ...",The IPO prospectus contains forward-looking st...,"Based on the provided content, the basis for t...","Based on the content provided, it is not possi...","Based on the content provided, we can analyze ...","Based on the provided content, the company's p...","According to the contents provided, the compan...",The provided content does not explicitly menti...,"Based on the provided Red Herring Prospectus, ...","Based on the provided contents, the shareholdi..."
4,750,https://www.chittorgarh.com/ipo/ifl-enterprise...,ifl enterprises,IFL Enterprises Ltd IPO,https://www.chittorgarh.com/ipo_subscription/i...,NaN,2017,2016,2017-03-07,9,...,"Based on the Red Herring Prospectus, the poten...","Based on the contents provided, the IPO price ...",The Issue Price of ₹56/- per Equity Share has ...,Based on the information provided in the Red H...,"Based on the provided content, the company bel...","Based on the content provided, here's an analy...","Based on the contents provided, it can be infe...","Based on the provided content, there is no inf...",Based on the information provided in the Red H...,"Based on the Red Herring Prospectus (RHP), the..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,451,https://www.chittorgarh.com

In [4]:
train = sme[sme['Close Year'] < 2023]
test = sme[sme['Close Year'] == 2023]

In [5]:
text_features = ["full_text_content",'answer_of_question_1',	'answer_of_question_2',	'answer_of_question_3','answer_of_question_4','answer_of_question_5','answer_of_question_6','answer_of_question_7','answer_of_question_8','answer_of_question_9','answer_of_question_10','answer_of_question_11','answer_of_question_12','answer_of_question_13','answer_of_question_14','answer_of_question_15','answer_of_question_16','answer_of_question_17','answer_of_question_18','answer_of_question_19','answer_of_question_20','answer_of_question_21','answer_of_question_22','answer_of_question_23','answer_of_question_24','answer_of_question_25']

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

In [7]:
import os
os.environ["WANDB_DISABLED"] = "true"

import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import os
import shutil

output_folder = "probability_outputs"
os.makedirs(output_folder, exist_ok=True)

full_df = sme.copy()

start = time.time()

# Preprocessing to handle NaN entries in the text columns
#for idx, feature in enumerate(tqdm(text_features, desc="Processing Features"), 1):
for idx, feature in enumerate(tqdm(text_features[(start_from-1):], desc="Processing Features"), start_from):
    if (time.time() - start) < (11.5*60*60):
        # Replace NaN entries with an empty string (you can modify this if you want a different placeholder)
        train[feature].fillna('', inplace=True)
        train_data = pd.DataFrame({'text': train[feature], 'labels': train[y]})
        train_dataset = Dataset.from_pandas(train_data)
        train_dataset = train_dataset.map(preprocess_function, batched=True)

        # test[feature].fillna('', inplace=True)
        # test_data = pd.DataFrame({'text': test[feature], 'labels': test[y]})
        # test_dataset = Dataset.from_pandas(test_data)
        # test_dataset = test_dataset.map(preprocess_function, batched=True)

        full_df[feature].fillna('', inplace=True)

        full_data = pd.DataFrame({'text': full_df[feature], 'labels': full_df[y]})
        full_dataset = Dataset.from_pandas(full_data)
        full_dataset = full_dataset.map(preprocess_function, batched=True)

        model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=2)

        training_args = TrainingArguments(
            output_dir=f"./results-{feature}",          # Output directory (can be ignored, we don't save models)
            evaluation_strategy="no",                   # No evaluation during training
            learning_rate=2e-5,                         # Learning rate
            per_device_train_batch_size=8,              # Batch size for training
            per_device_eval_batch_size=8,               # Batch size for evaluation
            num_train_epochs=3,                         # Number of epochs
            weight_decay=0.01,                          # Strength of weight decay
            logging_dir=None,                           # No logging
            save_total_limit=0,                         # No saving of checkpoints
            save_steps=0,                               # No saving steps
            load_best_model_at_end=False                # No need to load best model
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            tokenizer=tokenizer
        )

        with tqdm(total=training_args.num_train_epochs, desc=f"Training for feature '{feature}'", leave=False) as pbar:
            for epoch in range(training_args.num_train_epochs):
                trainer.train()
                pbar.update(1)  # Update progress bar for each epoch

        predictions = trainer.predict(full_dataset)
        preds_proba = torch.softmax(torch.tensor(predictions.predictions), dim=-1).numpy()

        feature_probabilities_df = pd.DataFrame({
            f'{feature}_class_0_prob': preds_proba[:, 0],  # Probability for class 0
            f'{feature}_class_1_prob': preds_proba[:, 1]   # Probability for class 1
        })

        

        output_file = os.path.join(output_folder, f'classification_probabilities_{idx}.csv')
        feature_probabilities_df.to_csv(output_file, index=False)
        print(f"Probabilities for feature '{feature}' saved to {output_file}")

        preds_labels = preds_proba.argmax(axis=1)
        true_labels = full_df[y].values
        accuracy = accuracy_score(true_labels, preds_labels)
        print(f"Accuracy for feature '{feature}': {accuracy:.4f}")
        shutil.rmtree(f"./results-{feature}")
        
print("All feature probabilities have been processed and saved.")


Processing Features:   0%|          | 0/26 [00:00<?, ?it/s]

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

Map:   0%|          | 0/681 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).

Training for feature 'full_text_content':   0%|          | 0/3 [00:00<?, ?it/s]

...

Step,Training Loss



Training for feature 'answer_of_question_25': 100%|██████████| 3/3 [06:20<00:00, 127.08s/it]
                                                                                            

Probabilities for feature 'answer_of_question_25' saved to probability_outputs/classification_probabilities_26.csv
Accuracy for feature 'answer_of_question_25': 0.8634


Processing Features: 100%|██████████| 26/26 [2:53:55<00:00, 401.38s/it]

All feature probabilities have been processed and saved.
